In [ ]:
# !pip install ib_insync
# !pip install schedule
# !pip install ibapi

In [1]:
from ib_insync import *
import requests
import pandas as pd
import schedule
import time
import datetime;
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)
# https://github.com/PacktPublishing/Python-for-Algorithmic-Trading-Cookbook/issues/7

In [3]:
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

<IB connected to 127.0.0.1:7496 clientId=0>

In [ ]:
pending_stop_loss_orders = []

In [ ]:
# util.logToConsole()

# https://stackoverflow.com/questions/23944657/typeerror-method-takes-1-positional-argument-but-2-were-given-but-i-only-pa
def order_status(self, trade):

    """
    print(trade)

    ||
    Fill(
        contract=Stock(conId=4727244, symbol='BMRA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='BMRA', tradingClass='SCM'), 
        
        execution=Execution(execId='00014f4f.6789209c.01.01', time=datetime.datetime(2025, 1, 16, 17, 42, 12, tzinfo=datetime.timezone.utc), 
        acctNumber='U12848716', exchange='IBKRATS', side='BOT', shares=1.0, price=0.6764, permId=239517332, clientId=0, orderId=129, liquidation=0, 
        cumQty=1.0, avgPrice=0.6764, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), 
        
        commissionReport=CommissionReport(execId='', commission=0.0, currency='', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), 
        time=datetime.datetime(2025, 1, 16, 17, 42, 12, 493388, tzinfo=datetime.timezone.utc))
    ||
    """

    print(trade) # works!! trade IS THE FILL
    print("and now")
    tr = ib.trades()
    print(tr[-1])
    
    """
    Fill(
    contract=Stock(conId=4727244, symbol='BMRA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='BMRA', tradingClass='SCM'), 
    
    execution=Execution(execId='00030f00.6789a4b7.01.01', time=datetime.datetime(2025, 1, 16, 17, 55, 39, tzinfo=datetime.timezone.utc), 
    acctNumber='U12848716', exchange='DARK', side='BOT', shares=1.0, price=0.6986, permId=239517344, clientId=0, orderId=153, liquidation=0, 
    cumQty=1.0, avgPrice=0.6986, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), 
    
    commissionReport=CommissionReport(execId='', commission=0.0, currency='', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), 
    time=datetime.datetime(2025, 1, 16, 17, 55, 39, 325899, tzinfo=datetime.timezone.utc))

    and now

    Trade(
    contract=Stock(conId=4727244, symbol='BMRA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='BMRA', tradingClass='SCM'), 
    
    order=LimitOrder(orderId=154, action='BUY', totalQuantity=1, lmtPrice=0.69), 
    
    orderStatus=OrderStatus(orderId=154, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, 
    clientId=0, whyHeld='', mktCapPrice=0.0), 
    
    fills=[], 
    
    log=[TradeLogEntry(time=datetime.datetime(2025, 1, 16, 17, 55, 39, 215073, tzinfo=datetime.timezone.utc), 
    status='PendingSubmit', message='', errorCode=0)], advancedError='')
    """



    
    """
        pending_stop_loss_orders.append(new_row)
        
                        
        contract = Stock('TSLA', 'SMART', 'USD')
        stopOrder = StopOrder('BUY', fill.execution.shares, 1.10)
        stopOrderTrade = ib.placeOrder(contract, stopOrder)
        #stopOrderTrade         
    """        
                
                
# test orders with day high volatility stocks
sym = 'BMRA'
test_symbols = [sym, sym]

for test in test_symbols:
    
    contract = Stock(test, 'SMART', 'USD')

    ib.qualifyContracts(contract)
    data = ib.reqMktData(contract)
    ib.sleep(2) # data.marketPrice() doesn't work without ib.sleep (few seconds)
    current_price = data.marketPrice()
    
    limitOrderUp = LimitOrder('BUY', 1, round(current_price * 1.015, 2))    
    limitOrderTradeUp = ib.placeOrder(contract, limitOrderUp)

    limitOrderDown = LimitOrder('BUY', 1, round(current_price * 0.985, 2))    
    limitOrderTradeDown = ib.placeOrder(contract, limitOrderDown)
    # limitOrderTrade contains the order and everything related to it, such as order status, fills and a log.
    # It will be live updated with every status change or fill of the order.
        
    # CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
    new_row = [limitOrderTradeUp.contract.symbol, limitOrderTradeUp.order.totalQuantity, limitOrderTradeUp.order.orderId, round(current_price * 1.25, 2)]
    pending_stop_loss_orders.append(new_row)

    new_row = [limitOrderTradeDown.contract.symbol, limitOrderTradeDown.order.totalQuantity, limitOrderTradeDown.order.orderId, round(current_price * 1.25, 2)]
    pending_stop_loss_orders.append(new_row)

    
    #new_row = pd.DataFrame({[limitOrderTrade.contract.symbol], [limitOrderTrade.order.totalQuantity], 
    #                        [limitOrderTrade.order.orderId], [round(current_price * 1.25, 2)]})
    #pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)

    # fillEvent (trade: Trade, fill: Fill)
    limitOrderTradeUp.fillEvent += order_status
    limitOrderTradeDown.fillEvent += order_status

In [13]:
trads = ib.trades()
trads

[Trade(contract=Stock(conId=4727244, symbol='BMRA', right='?', exchange='SMART', currency='USD', localSymbol='BMRA', tradingClass='SCM'), order=Order(orderId=134, permId=239517335, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=0.67, auxPrice=0.0, tif='DAY', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U12848716', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=134, status='Submitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=239517335, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 1, 16, 18, 5, 37, 379415, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0)], advancedError=''),
 Trade(contract=Stock(conId=4727244, symbol='BMRA', right='?', exchange='SMART', currency='USD', localSymbol='

In [ ]:
ords = ib.orders()
ords

In [ ]:
trades = ib.trades()
trades

In [ ]:
pending_stop_loss_orders = [] # define and create one time
# https://stackoverflow.com/questions/10487278/how-to-declare-and-add-items-to-an-array-in-python

In [ ]:
new_row = ["BABY", 1, 0.15, 99]
pending_stop_loss_orders.append(new_row)
new_row = ["COME", 1, 0.35, 79]
pending_stop_loss_orders.append(new_row)
new_row = ["HOME", 1, 0.55, 59]
pending_stop_loss_orders.append(new_row)

In [ ]:
pending_stop_loss_orders

In [ ]:
del pending_stop_loss_orders[1]

In [ ]:
pending_stop_loss_orders